# Characteristic Stability Index (CSI) as a statistic

<img src="thumbpic.png" alt="UK with cities of interest marked" width="500"/>

Characteristic Stability Index (CSI), and the closely related Population Stability Index (PSI) are amongest recognized metrics to measure the distribution stability for numeric features.  Commonly CSI is presented as a universal metric with rule-of-thumb thresholds that are understood to have wide applicability. In this post I will instead treat it as a statistic and demonstrate that sample size and the underlying randomness of the data source can drastically alter what one should count as large or small for CSI. I will also present a JAX-based Python library to estimate appropriate CSI thresholds.

## Introduction

For simplicity, we shall focus on a real-valued feature such as temperature *X*. Let there be the reference set of observations of *X*, which are recognized as 'normal', call it the 'left' set, and a new set of observations which need to be compared to normal, call that the 'right' set:

$$
\begin{align}
X^{(L)}&\to {X^{(L)}_1,\dots X^{(L)}_N},\quad left\,set \\
X^{(R)}&\to {X^{(R)}_1,\dots X^{(R)}_M},\quad right\,set \\
\end{align}
$$

One then bins the observations into, typically, 10 bins such that each bin contains 10% of the observations from the left set:

| bin number | bin lower bound | bin upper bound | number of left-set observations in bin                    | number of right-set observations in bin                    |
|------------|-----------------|-----------------|-----------------------------------------------------------|------------------------------------------------------------|
| 0          | $B_0$           | $B_1$           | $A^{(L)}_0 \approx N/10$                                  | $A^{(R)}_0$                                                |
| 1          | $B_1$           | $B_2$           | $A^{(L)}_1 \approx N/10$                                  | $A^{(R)}_1$                                                |
| ...        | ...             | ...             | ...                                                       | ...                                                        |
| 9          | $B_9$           | $B_{10}$        | $A^{(L)}_9 \approx N/10$                                  | $A^{(R)}_9$                                                |

**Note:** Throughout the text I shall refer to arrays of occupancy, the fourth and fifth columns in the table above as *histograms* to simplify text. 

The CSI computed from the relative occupancy of each bin, i.e. for $\rho^{(L)}_i=A^{(L)}_i/N$ and $\rho^{(R)}_i=A^{(R)}_i/M$ the CSI is:

$$
CSI=\sum_{i=0}^9 \left(\rho^{(L)}_i - \rho^{(R)}_i\right)\cdot\log\frac{\rho^{(L)}_i}{\rho^{(R)}_i}
$$

Sometimes $\rho^{(\dots)}_{\dots}$ is done as a percentage, which has the effect of multiplying CSI by 100. Often, one adds small number to each $\rho^{(\dots)}_{\dots}$ in order to prevent dividing by zero or computing logarithms of zero. 

For example, consider the temperature observations (12 months for the last century+) for [Oxford (UK)](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/oxforddata.txt) and [Durham (UK)](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/durhamdata.txt) available from the Met-Office UK (using `tmax degC` column). Breaking the the temperature into bins and collecting it into histograms:

| bin number | bin lower bound (degrees) | bin upper bound (degrees) | number of Oxford observations in this bin   | number of Durham observations in this bin     |
|------------|-----------------|---------------------------|---------------------------------------------|-----------------------------------------------|
| 0          | low             | 6.8                       | 198                                         | 269                                           |
| 1          | 6.8             | 8.4                       | 197                                         | 207                                           |
| 2          | 8.4             | 9.6                       | 197                                         | 218                                           |
| 3          | 9.6             | 11.5                      | 218                                         | 165                                           |
| 4          | 11.5            | 13.9                      | 206                                         | 212                                           |
| 5          | 13.9            | 16.0                      | 204                                         | 161                                           |
| 6          | 16.0            | 18.2                      | 196                                         | 244                                           |
| 7          | 18.2            | 19.9                      | 206                                         | 180                                           |
| 8          | 19.9            | 21.5                      | 208                                         | 92                                            |
| 9          | 21.5            | high                      | 205                                         | 32                                            |

One can quickly estimate that CSI between these two distributions would be around 0.23. Similar approach has been described in:
* https://parthaps77.medium.com/population-stability-index-psi-and-characteristic-stability-index-csi-in-machine-learning-6312bc52159d
* https://www.lexjansen.com/wuss/2017/47_Final_Paper_PDF.pdf
* https://towardsdatascience.com/psi-and-csi-top-2-model-monitoring-metrics-924a2540bed8
* https://www.linkedin.com/pulse/credit-risk-scorecard-monitoring-tracking-shailendra/
* https://www.lexjansen.com/wuss/2017/47_Final_Paper_PDF.pdf
* https://towardsdatascience.com/data-drift-part-2-how-to-detect-data-drift-1f8bfa1a893e

With some simple analysis one can relate the CSI to [Kullback-Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence). More, precisely, CSI is a symmetrized version of KL-divergence, e.g. see [SE what-is-the-intuition-behind-the-population-stability-index](https://stats.stackexchange.com/questions/219822/what-is-the-intuition-behind-the-population-stability-index)

## Treating CSI as a statistic

One way to get a better handle on CSI, is to note that it is based solely on the histograms of the left and right sets of observations, i.e. the actual sets of obervations are not important. Therefore one can reduce the problem to considering the expected CSI from a set of samples drawn from the [multinomial distributions](https://en.wikipedia.org/wiki/Multinomial_distribution). There can be left and right set of probabilities:

$$
\begin{align}
p^{(L)}&\to {p^{(L)}_0,\dots, p^{(L)}_9},\quad \sum_{i} p^{(L)}_i=1 \\
p^{(R)}&\to {p^{(R)}_0,\dots, p^{(R)}_9},\quad \sum_{i} p^{(R)}_i=1 \\
\end{align}
$$

One can then draw samples with different sizes from the 'left' and 'right' multinomial distributions, using the probabilities above, and compute the appropriate CSI-s. For example, let: 

$$
\begin{align}
p^{(L)}&\to {0.1,\dots, 0.1} \\
p^{(R)}&\to {0.4, 0.3, 0.2, 0.1, 0.0,\dots, 0.0} \\
\end{align}
$$

for 10 categpories. 

One can obtain these using, for example:

```python
import numpy as np
import numpy.random as npr

cat_count=10 # number of categories (bins in the histogram)
bin_bounds = np.array([-1, *range(cat_count)])+0.5 # prepare bin boundaries, e.g. [-0.5, 0.5, 1.5, ...]
eps = 1e-3 # extra to add to force non-zero in all bins

# generate the left histogram, e.g. getting 
# `left_hist = [47.001, 51.001, 38.001, 49.001, 46.001, 46.001, 49.001, 44.001, 42.001, 38.001]`
left_count=450;
left_hist = np.histogram(
    npr.choice(
        range(cat_count),
        replace=True,
        size=left_count,
        p=[1./cat_count]*cat_count
    ),
    bins=bin_bounds
)[0]+eps
#
norm_left_hist = left_hist/sum(left_hist)

# generate right histogram, e.g. getting
# `right_hist = [142.001, 112.001, 85.001, 41.001, 1e-3, ..., 1e-3])`
right_count=380;
right_hist = np.histogram(
    npr.choice(
        range(cat_count),
        replace=True,
        size=right_count,
        p=[0.4, 0.3, 0.2, 0.1, *[0.0]*(cat_count-4)]
    ),
    bins=bin_bounds
)[0]+eps
norm_right_hist = right_hist/sum(right_hist)

trial_csi = np.sum((norm_left_hist-norm_right_hist)*np.log(norm_left_hist/norm_right_hist))
print(trial_csi)
```

Which ends up giving extremely large value, e.g. 4...8 depending on `eps`. 

The actual value is not important, what matters is that:

**The expectations for CSI, i.e. the null hypothesis, can be expressed in terms of the expectations for the multinomial distributions that give rise to histograms which then lead to CSI.**


### Null hypothesis

A simple null hypothesis would be that left and right histograms come from the same multinomial distribution, perhaps with equal probabilities of each category. This however misses the noise inherent in the system under inverstigation. For example, with Met-Office temperature measurements used in preceeeding section, the temperatures in the source data are specified to within one decimal point of a degree. This would mean that 20.06 and 21.14 would be rounded up to the same value of 21.1 degrees, so one has to expect the inherent noise in the temperature be on the order of 0.04 degrees plus/minus.If the bins for the temperature are about 2 degrees wide, then the change in width of the bin due to noise can be 1.92...2.08 degrees i.e. 8% change.

One way to capture this is to assume that left and right histograms come from the same multinomial distribution, but the probabilities of each category are not precise, instead they are themselves random numbers with a known distribution. A convenient parametrization here is to express probabilities of all categories as logits, and describe all logits as normally distributed random variables from a distribution with known mean and an unknown variance:

$$
\begin{align}
K&:\:number\,of\,categories \\
s&:\:standard\,deviation\,in\,the\,logits \\
\mu&=\mbox{logit}\left(1/K\right)=-\log\left(K-1\right) \\
l_i & \sim N\left(\mu,\,s^2\right),\quad i=0,\dots,K\,\quad logits\,for\,category\,probabilities \\
p_i &=\sigma\left(l_i\right)=\left(1+\exp\left(-l_i\right)\right)^{-1},\quad probabilities\,for\,categories
\end{align}
$$

Whilst this may seem somewhat complex, it is quite simple to implement

```python
import numpy as np
import numpy.random as npr

import scipy as sp
import scipy.stats as sp_st
import scipy.special as sp_spec

# set up constants
cat_count = 10 # K - number of categories (bins in the histogram)
logit_std = 0.01 # s - standard deviation for logits
sample_count = 400 # number of samples to draw from the multinomial distribution
#
logit_mu = sp_spec.logit(1./cat_count) # logit mean for all categories

####
def draw_random_histogram():
    # draw a set of probabilities for the categories
    logit_arr = logit_mu + npr.normal(size=cat_count) * logit_std
    pval_arr = sp_spec.expit(logit_arr)
    pval_arr /= sum(pval_arr) # normalize to sum to 1

    # draw the samples from the categorical distribution
    # and extract them as a histogram straight away
    random_histogram = sp_st.multinomial.rvs(n=sample_count, p=pval_arr)
    
    return random_histogram

####

# draw left and right histograms
left_hist = draw_random_histogram()
right_hist = draw_random_histogram()

### compute CSI
norm_left_hist = left_hist / sum(left_hist)
norm_right_hist = right_hist / sum(right_hist)
csi = sum((norm_left_hist - norm_right_hist)*np.log(norm_left_hist/norm_right_hist))

print('left histogram: ', left_hist)
print('right histogram: ', right_hist)
print(f'csi: {csi:.3e}')
```

by adjusting `logit_std` and `sample_count` one can easily see that larger number of samples does decrease the CSI, but as one increases it, the CSI usually hits a limit, which is dictated by `logit_std`. 

As we will demonstrate in the next section, by repeatedly sampling CSI, in a fashion shown here one can build what is essentially a hypothesis test, where CSI is treated as a statistic.

## Worked example: CSI for max monthly temperatures for different UK locations

As an example, we shall return to historical monthly temperatures provided by the Met-Office UK. Here are the locations we shall consider:

* [Stornoway](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/stornowaydata.txt)
* [Armagh](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/armaghdata.txt)
* [Durham](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/durhamdata.txt)
* [Bradford](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/bradforddata.txt)
* [Sheffield](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/sheffielddata.txt)
* [Oxford](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/oxforddata.txt)
* [Southampton](https://www.metoffice.gov.uk/pub/data/weather/uk/climate/stationdata/southamptondata.txt)

<img src="uk_schematic.png" alt="UK with cities of interest marked" width="350"/>

Southampton, as the southern-most city will be the reference point, i.e. the grid for the histograms will be such that monthly Southampton's temperatures would be split into roghly the same-sized 10 bins.

The full code for loading the data and splitting it into bins is provided in [a separate notebook notebook](extract_weather_patterns_v2.ipynb), here we shall only present illustration of the histograms extracted for Southampton.

| histogram index | time-period | T < 7.6 | (7.6...9.0) | (9.0...10.2) | (10.2...12.0) | (12.0...14.25) | (14.25...16.4) | (16.4...18.4) | (18.4...20.0) |  (20.0...21.4) | T>21.4 |
|---|---|---|---|---|---|---|---|---|---|---|---|
| 0 | 1855-1864 | 13 | 17 | 11 |  8 | 10 | 13 | 15 | 11 |  9 | 13 |
| 1 | 1865-1974 | 17 | 10 | 10 | 13 | 12 | 13 | 9 | 11 | 11 | 14 |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... |
| 12 | 1978-1987 | 12 | 13 | 12 | 13 | 8 | 16 | 9 | 17 | 9 | 11 |

So, for example, in the 10-year span 1978-1987, corresponding to histogram #12, 12 maximum monthly temperatures were below 7.6 degrees (Centigrade), 13 observations were between 7.6 and 9.0 degrees etc. Still staying with the #12 histogram, 12 out of 120 observations were below 7.6 degrees, this corresponds to proportion of 0.1 and logit of -2.197=1/(1+exp(-0.1)). Converting all observations into logits in this manner, for all locations, and then computing the mean logit and the standard deviation one finds:

| location     | mean logit | logit standard deviation |
|--------------|------------|--------------------------|
| Stornoway    | -4.086     | 3.393                    |
| Armagh       | -2.504     | 3.393                    |
| Durham       | -2.503     | 1.198                    |
| Bradford     | -2.532     | 1.166                    |
| Sheffield    | -2.363     | 0.678                    |
| Oxford       | -2.254     | 0.383                    |
| Southampton  | -2.228     | 0.282                    |

Unsurprisingly, Southampton's logits are closest to -2.2 and with lowest standard deviation, which corresponds to all obersvations being split roughly equally between the 10 bins. This is direct consequence on choosing bin boundaries based on Southampton as a reference point.

Next, one can take the 10-year aggregated histograms for different locations and compare them with Southampton's by computing CSI. Using the most recent 10-year histograms in all cases one finds:

| left histogram's location | right histogram's location | CSI   |
|---------------------------|----------------------------|-------|
| Southampton               | Stornoway                  | 3.29  |
| Southampton               | Armagh                     | 0.36  |
| Southampton               | Durham                     | 0.35  |
| Southampton               | Bradford                   | 0.46  |
| Southampton               | Sheffield                  | 0.24  |
| Southampton               | Oxford                     | 0.08  |

Which CSI is large enough to take as evidence of substantially different distribution of monthly maximum temperatures? Rule of thumb for CSI is to take anything above 0.2 as significant change, however this rule of thumb fails to take into account the noise in the data.

Instead, one can adopt the null-hypothesis as left and right histograms coming from multinomial distribution where probability of each category corresponds to the sigma-function of a logit, whilst the logit is a normally distributed random variable with mean of -2.197 and standard deviation of -0.282 (comes from Southampton data). Drawing 300,000 of pairs of such histograms, with 120 samples in each (10 years of 12 months) one can find the distribution of the expected CSI (see [a separate notebook notebook](extract_weather_patterns_v2.ipynb)). Using this, one can estimate that 95% of all observed CSIs will be below 0.453. 

*Therefore, if one was to treat this as a hypothesis test, at 95% confidence, one would reject the null hypothesis only for Bradford and Stornoway (since there the CSI is above 0.45). Observations for all other locations do not differ sufficiently from the distribution of temperatures for Southampton.*

### Specifics of implementation

Drawing a large sample of CSIs, to establish thresholds, as done above, is, in principle, simple, but can be quite time-consuming if one has to cycle. A much better solution is to vectorize this process. Correspondingly a JAX-based implementation  for drawing CSI samples is offered as part of this post ([csi_calc.draw_csi_with_logit_variance](csi_calc.py)). The useage is as follows (see a separate notebook notebook)

```python
...
import jax.config as jc
jc.update('jax_enable_x64', True)
import jax
import jax.numpy as jn
import jax.scipy as js
import jax.random as jr
import csi_calc as cc
...

base_logits = jn.array([1./cat_count]*cat_count)
logit_std = agg_geos_df.query(f'location=="{hist_choice_location}"').logit_std.values[0]
pop_count = chunk_size_years * 12 # number of measurements in each histogram (12 per year)

csi_draw_count = 300000 # number of CSI's to compute

rnd_key, csi_rnd_key = jr.split(rnd_key, num=1+1)
tick = dt.datetime.now()
#
csi_samples = cc.draw_csi_with_logit_variance(
    rnd_key=csi_rnd_key,
    base_logits=base_logits,
    logits_std=logit_std,
    sample_count=csi_draw_count,
    count_left=pop_count,
    count_right=pop_count
)
#
tock = dt.datetime.now()
time_delta = tock-tick
time_delta_sec = time_delta.seconds + time_delta.microseconds * 1e-6

highq = 0.95
highq_csi = jn.quantile(csi_samples, q=highq)

print(f'Time taken: {time_delta_sec:.3f} sec for {csi_draw_count} samples.')
print(f'Quantile q={highq:.3f} corresponds to CSI={highq_csi:.3e}')
```

The calculation takes 10-20 sec on an average laptop, and can effectively use available cores and RAM.

## Conclusion

Characteristic Stability Index can be a useful way for estimating the drift in the distribution of a random variable, given a test set and a reference set. To draw informative conclusions it is best avoid using rule-of-thumb thresholds, as these ignore:

* Size of your dataset
* Inherent noise in your dataset

Instead, one can estimate appropriate thresholds by building a sound Null Hypothesis and then estimating the expected distribution of CSI under this Null Hypothesis. Modern tools such as JAX make this a relatively inexpensive and scaleable exercise.